In [11]:
import pandas as pd
import os

These are the datasets available:

In [12]:
print(os.listdir('flow data/flow data'))

['usgs_san_francisquito.txt', 'usgs_data.csv', 'influent_flow.csv', 'weather.csv']


The usgs_data.csv is just the table from usgs_san_francisquito.txt

Let's take a look at each dataset and see if any cleaning/manipulation needs to be done

In [13]:
#Looking at influent_flow.csv table, our target
flow = pd.read_csv('flow data/flow data/influent_flow.csv')
print('Rows: ', flow.shape[0], '\nColumns: ', flow.shape[1])
print(flow.info())

#Change the date column to a datetime for this table as well
flow['Date'] = pd.to_datetime(flow['Date'])

flow.head()

Rows:  34516 
Columns:  2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34516 entries, 0 to 34515
Data columns (total 2 columns):
Date        34516 non-null object
INF_FLOW    34516 non-null float64
dtypes: float64(1), object(1)
memory usage: 539.4+ KB
None


,Date,INF_FLOW
0,2013-01-15 00:00:00,9.589273
1,2013-01-15 01:00:00,9.272833
2,2013-01-15 02:00:00,8.861329
3,2013-01-15 03:00:00,9.193454
4,2013-01-15 04:00:00,9.129482


In [14]:
#Looking at weather table
weather = pd.read_csv('flow data/flow data/weather.csv')
print('Rows: ', weather.shape[0], '\nColumns: ', weather.shape[1])
print(weather.info())

#Change the date column to a datetime data type for easier manipulation
weather['Date'] = pd.to_datetime(weather['Date'])

weather.head()

Rows:  44784 
Columns:  8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44784 entries, 0 to 44783
Data columns (total 8 columns):
Date               44784 non-null object
rain               44784 non-null float64
snow               44784 non-null float64
accum              44784 non-null float64
temp               44784 non-null float64
humidity           44784 non-null float64
cloud              44784 non-null float64
app_temperature    44784 non-null float64
dtypes: float64(7), object(1)
memory usage: 2.7+ MB
None


,Date,rain,snow,accum,temp,humidity,cloud,app_temperature
0,2012-12-15 08:00:00,0.0,0.0,0.0,5.95,0.89,0.08,5.95
1,2012-12-15 09:00:00,0.0,0.0,0.0,5.66,0.92,0.09,5.66
2,2012-12-15 10:00:00,0.0,0.0,0.0,5.46,0.91,0.09,5.46
3,2012-12-15 11:00:00,0.0,0.0,0.0,4.82,0.92,0.06,4.82
4,2012-12-15 12:00:00,0.0,0.0,0.0,4.61,0.93,0.03,4.61


In [15]:
#Looking at usgs_san_francisuito.txt table
usgs = pd.read_csv('flow data/flow data/usgs_data.csv', sep = '\t', low_memory = False)
print('Rows: ', usgs.shape[0], '\nColumns: ',usgs.shape[1])
print(usgs.info()) #Looks like we have some missing values in the last 4 columns

#get rid of row 0 (indicating datatype and length for each column)
usgs.drop(0, inplace= True)

#Change the date column to a datetime for this table as well
usgs['datetime'] = pd.to_datetime(usgs['datetime'])

#The original usgs text file indicates the 14747_00060 and 14748_0065 columns 
#indicate discharge and gage height. Lets change the name of those columns
#and their corresponding qualification codes. Let's also rename the 'datetime'
#to 'Date' to be consitent with the other tables

usgs.rename(columns = {'14747_00060': 'discharge', 
            '14747_00060_cd': 'discharge_cd',
            '14748_00065' : 'gage_height',
            '14748_00065_cd' : 'gage_height_cd',
            'datetime' : 'Date'},
           inplace = True)
print(usgs.discharge_cd.unique(), usgs.gage_height_cd.unique())
usgs.head()

Rows:  178896 
Columns:  8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178896 entries, 0 to 178895
Data columns (total 8 columns):
agency_cd         178896 non-null object
site_no           178896 non-null object
datetime          178896 non-null object
tz_cd             178896 non-null object
14747_00060       178676 non-null object
14747_00060_cd    178676 non-null object
14748_00065       178761 non-null object
14748_00065_cd    178761 non-null object
dtypes: object(8)
memory usage: 10.9+ MB
None
['A' nan 'P:e' 'A:e' 'P'] ['A' nan 'P']


,agency_cd,site_no,Date,tz_cd,discharge,discharge_cd,gage_height,gage_height_cd
1,USGS,11164500,2013-01-01 00:00:00,PST,44.1,A,1.27,A
2,USGS,11164500,2013-01-01 00:15:00,PST,44.1,A,1.27,A
3,USGS,11164500,2013-01-01 00:30:00,PST,44.1,A,1.27,A
4,USGS,11164500,2013-01-01 00:45:00,PST,43.2,A,1.26,A
5,USGS,11164500,2013-01-01 01:00:00,PST,43.2,A,1.26,A


In [16]:
print(usgs.discharge_cd.unique(), usgs.gage_height_cd.unique())
# usgs[(usgs['discharge_cd'] !='A') | (usgs['gage_height_cd'] != 'A')]

['A' nan 'P:e' 'A:e' 'P'] ['A' nan 'P']


Now that the 3 dataframes have been cleaned up a bit lets merge them together by date

In [53]:
df = flow.merge(weather)
df = df.merge(usgs)
df.info()
print(df.discharge_cd.unique(), df.gage_height_cd.unique())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34324 entries, 0 to 34323
Data columns (total 16 columns):
Date               34324 non-null datetime64[ns]
INF_FLOW           34324 non-null float64
rain               34324 non-null float64
snow               34324 non-null float64
accum              34324 non-null float64
temp               34324 non-null float64
humidity           34324 non-null float64
cloud              34324 non-null float64
app_temperature    34324 non-null float64
agency_cd          34324 non-null object
site_no            34324 non-null object
tz_cd              34324 non-null object
discharge          34269 non-null object
discharge_cd       34269 non-null object
gage_height        34323 non-null object
gage_height_cd     34323 non-null object
dtypes: datetime64[ns](1), float64(8), object(7)
memory usage: 4.5+ MB
['A' nan 'P:e'] ['A' nan]


In [54]:
#Since we have a relatively small amount of null values I'll assume it's okay to drop their rows for now rather 
#than try to impute their values. Even if there's an MAR affect happening, it likely wont have an impact on the
#results given that values with null rows only take up <0.2% of the data
df.dropna(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34268 entries, 0 to 34323
Data columns (total 16 columns):
Date               34268 non-null datetime64[ns]
INF_FLOW           34268 non-null float64
rain               34268 non-null float64
snow               34268 non-null float64
accum              34268 non-null float64
temp               34268 non-null float64
humidity           34268 non-null float64
cloud              34268 non-null float64
app_temperature    34268 non-null float64
agency_cd          34268 non-null object
site_no            34268 non-null object
tz_cd              34268 non-null object
discharge          34268 non-null object
discharge_cd       34268 non-null object
gage_height        34268 non-null object
gage_height_cd     34268 non-null object
dtypes: datetime64[ns](1), float64(8), object(7)
memory usage: 4.4+ MB


In [55]:
#Check which columns have only 1 unique value. If that's the case, they won't be able to have an effect and 
#can be dropped to simplify our dataframe further

unique_value_cols = [col for col in df.columns if df[col].unique().size == 1]
unique_value = [df[col].unique()[0] for col in df.columns if df[col].unique().size == 1]
print('Column name: \t' ,unique_value_cols,'\nUnique Value\t', unique_value)

Column name: 	 ['snow', 'accum', 'agency_cd', 'site_no', 'discharge_cd', 'gage_height_cd'] 
Unique Value	 [0.0, 0.0, 'USGS', '11164500', 'A', 'A']


In [62]:
df.drop(unique_value_cols, axis = 1, inplace = True)

In [63]:
df.head()

,Date,INF_FLOW,rain,temp,humidity,cloud,app_temperature,tz_cd,discharge,gage_height
0,2013-01-15 00:00:00,9.589273,0.0,10.19,0.26,0.04,10.19,PST,17.1,0.86
1,2013-01-15 01:00:00,9.272833,0.0,9.76,0.31,0.18,9.76,PST,17.1,0.86
2,2013-01-15 02:00:00,8.861329,0.0,7.80,0.36,0.07,6.92,PST,17.1,0.86
3,2013-01-15 03:00:00,9.193454,0.0,6.52,0.43,0.03,6.52,PST,17.1,0.86
4,2013-01-15 04:00:00,9.129482,0.0,5.69,0.48,0.03,5.69,PST,16.7,0.85
